## NLP for ML Classification

**Hypothesis**: Part of Speech (POS) tagging and syntactic dependency parsing provides valuable information for classifying imperative phrases. The thinking is that being able to detect imperative phrases will transfer well to detecting tasks and to-dos.

#### Some Terminology
- [_Imperative mood_](https://en.wikipedia.org/wiki/Imperative_mood) is "used principally for ordering, requesting or advising the listener to do (or not to do) something... also often used for giving instructions as to how to perform a task."
- _Part of speech (POS)_ is a way of categorizing a word based on its syntactic function.
    - The POS tagger from Spacy.io that is used in this notebook differentiates between [*pos_* and *tag_*](https://spacy.io/docs/api/annotation#pos-tagging-english) - *POS (pos_)* refers to "coarse-grained part-of-speech" like `VERB`, `ADJ`, or `PUNCT`; and *POSTAG (tag_)* refers to "fine-grained part-of-speech" like `VB`, `JJ`, or `.`.
- _Syntactic dependency parsing_ is a way of connecting words based on syntactic relationships, [such as](https://spacy.io/docs/api/annotation#dependency-parsing-english) `DOBJ` (direct object), `PREP` (prepositional modifier), or `POBJ` (object of preposition).
    - Check out the dependency parse for the phrase ["Send the report by Kyle by tomorrow"](https://demos.explosion.ai/displacy/?text=Send%20the%20report%20by%20Kyle%20by%20tomorrow&model=en&cpu=1&cph=1) as an example

#### Features
The imperative mood centers around _actions_, and actions are generally represented in English using verbs. So the features are engineered to also center on the VERB:
1. *FeatureName.VERB*: Does the phrase contain VERB(s) of the tag form VB*?
2. *FeatureName.FOLLOWING_POS*: Are the words following the VERB(s) of certain parts of speech?
3. *FeatureName.FOLLOWING_POSTAG*: Are the words following the VERB(s) of certain POS tags?
4. *FeatureName.CHILD_DEP*: Are the VERB(s) parents of certain syntactic dependencies?
5. *FeatureName.PARENT_DEP*: Are the VERB(s) children of certain syntactic dependencies?
6. *FeatureName.CHILD_POS*: Are the syntactic dependencies that the VERB(s) are children of of certain parts of speech?
7. *FeatureName.CHILD_POSTAG*: Are the syntactic dependencies that the VERB(s) are children of of certain POS tags?
8. *FeatureName.PARENT_POS*: Are the syntactic dependencies that the VERB(s) parent of certain parts of speech?
9. *FeatureName.PARENT_POSTAG*: Are the syntactic dependencies that the VERB(s) parent of certain POS tags?

Note that features 2-9 all depend on feature 1 between `True`; if `False`, phrase vectorization will result in all zeroes.

## Data and Setup

### Building a recipe corpus

I wrote and ran `epicurious_recipes.py`\* to scrape Epicurious.com for recipe instructions and descriptions. Output is `epicurious-pos.txt` and `epicurious-neg.txt`.

\* _script (very) loosely based off of https://github.com/benosment/hrecipe-parse_

Note that deriving all negative examples in the training set from Epicurious recipe descriptions would result in negative examples that are longer and syntactically more complicated than the positive examples. This is a form of bias.

To (hopefully?) correct for this a bit, I will add the short movie reviews found at https://pythonprogramming.net/static/downloads/short_reviews/ as more negative examples.

This still feels weird because we're selecting negative examples only from specific categories of text (recipe descriptions, short movie reviews) - just because they're readily available.

Ultimately though, this recipe corpus is a **stopgap/proof of concept** for a corpus more relevant to tasks later on, so I won't worry further about this for now.

In [1]:
import os
import random

In [2]:
BASE_DIR = os.getcwd()
pos_data_path = BASE_DIR + '/pos.txt'
neg_data_path = BASE_DIR + '/neg.txt'

In [3]:
with open(pos_data_path, 'r', encoding='utf-8') as f:
    pos_data = f.read()
with open(neg_data_path, 'r', encoding='utf-8') as f:
    neg_data = f.read()

In [4]:
pos_data_split = pos_data.split('\n')
neg_data_split = neg_data.split('\n')

num_pos = len(pos_data_split)
num_neg = len(neg_data_split)

# 50/50 split between the number of positive and negative samples
num = num_pos if num_pos < num_neg else num_neg

# shuffle samples
random.shuffle(pos_data_split)
random.shuffle(neg_data_split)

In [5]:
lines = []
for l in pos_data_split[:num]:
    lines.append((l, 'pos'))
for l in neg_data_split[:num]:
    lines.append((l, 'neg'))

In [6]:
from enum import Enum, auto
class FeatureName(Enum):
    VERB = auto()
    FOLLOWING_POS = auto()
    FOLLOWING_POSTAG = auto()
    CHILD_DEP = auto()
    PARENT_DEP = auto()
    CHILD_POS = auto()
    CHILD_POSTAG = auto()
    PARENT_POS = auto()
    PARENT_POSTAG = auto()

## [spaCy.io](https://spacy.io/) for NLP
_Because Stanford CoreNLP is hard to install for Python_

Found Spacy through an article on ["Training a Classifier for Relation Extraction from Medical Literature"](https://www.microsoft.com/developerblog/2016/09/13/training-a-classifier-for-relation-extraction-from-medical-literature/) ([GitHub](https://github.com/CatalystCode/corpus-to-graph-ml))

<img src="nltk_library_comparison.png" alt="NLTK library comparison chart https://spacy.io/docs/api/#comparison" style="width: 400px; margin: 0;"/>

In [7]:
#!conda config --add channels conda-forge
#!conda install spacy
#!python -m spacy download en

### Using the Spacy Data Model for NLP

In [8]:
import spacy
nlp = spacy.load('en')

Spacy's sentence segmentation is lacking... https://github.com/explosion/spaCy/issues/235. So each '\n' will start a new Spacy Doc.

In [9]:
def create_spacy_docs(ll):
    dd = [(nlp(l[0]), l[1]) for l in ll]
    # collapse noun phrases into single compounds
    for d in dd:
        for np in d[0].noun_chunks:
            np.merge(np.root.tag_, np.text, np.root.ent_type_)
    return dd

In [10]:
docs = create_spacy_docs(lines)

### NLP output

Tokenization, POS tagging, and dependency parsing happened automatically with the `nlp(line)` calls above! So let's look at the outputs.

https://spacy.io/docs/usage/data-model and https://spacy.io/docs/api/doc will be useful going forward

In [11]:
for doc in docs[:10]:
    print(list(doc[0].sents))

[Divide mushrooms and chickpeas into separate piles and spread in a single layer.]
[Add 1 Tbsp. oil, then sugar snap peas, and shake pan so sugar snaps spread out in a single layer.]
[Line a baking sheet with parchment paper.]
[Let sit at room temperature 15 minutes before cooking.]
[Tip into a bowl with the ground almonds and flour and mix together.]
[After cooled, cut fritters into pieces to top cupcakes.]
[Flip the chicken over and cook for 5 minutes more.]
[In a second shallow bowl, combine the polenta, cheese, salt, and pepper.]
[Drizzle with oil; season with salt and pepper.]
[Roast, tossing occasionally, until golden brown, 10–12 minutes.]


In [12]:
for doc in docs[:10]:
    print(list(doc[0].noun_chunks))

[Divide mushrooms, chickpeas, separate piles, a single layer]
[oil, pan, so sugar snaps, a single layer]
[Line, a baking sheet, parchment paper]
[room temperature, cooking]
[Tip, a bowl, the ground almonds, flour]
[fritters, pieces, top cupcakes]
[the chicken, 5 minutes]
[a second shallow bowl, the polenta]
[Drizzle, oil, season, salt, pepper]
[Roast, golden brown]


[Spacy's dependency graph visualization](https://demos.explosion.ai/displacy)

In [13]:
for doc in docs[:5]:
    for token in doc[0]:
        print(token.text, token.dep_, token.lemma_, token.pos_, token.tag_, token.head, list(token.children))

Divide mushrooms nsubj Divide mushrooms NOUN NNS spread [and, chickpeas, into]
and cc and CCONJ CC Divide mushrooms []
chickpeas conj chickpeas NOUN NNS Divide mushrooms []
into prep into ADP IN Divide mushrooms [separate piles]
separate piles pobj separate piles NOUN NNS into [and]
and cc and CCONJ CC separate piles []
spread ROOT spread VERB VB spread [Divide mushrooms, in, .]
in prep in ADP IN spread [a single layer]
a single layer pobj a single layer NOUN NN in []
. punct . PUNCT . spread []
Add ROOT add VERB VB Add [1, Tbsp, ., oil, then, peas, and, shake, spread, .]
1 nummod 1 NUM CD Add []
Tbsp appos tbsp PROPN NNP Add []
. punct . PUNCT . Add []
oil dobj oil NOUN NN Add [,]
, punct , PUNCT , oil []
then advmod then ADV RB Add []
sugar compound sugar NOUN NN snap []
snap compound snap NOUN NN peas [sugar]
peas dep pea NOUN NNS Add [snap, ,]
, punct , PUNCT , peas []
and cc and CCONJ CC Add []
shake conj shake VERB VB Add [pan]
pan dobj pan NOUN NN shake []
so sugar snaps nsubj s

### Featurization

In [14]:
import re
from collections import defaultdict

def featurize(d):
    s_features = defaultdict(int)
    for idx, token in enumerate(d):
        #print(token, token.pos_, token.tag_)
        if re.match(r'VB.?', token.tag_) is not None: # note: not using token.pos == VERB because this also includes BES, HVS, MD tags 
            s_features[FeatureName.VERB.name] += 1
            # FOLLOWING_POS
            next_idx = idx + 1;
            if next_idx < len(d):
                s_features[f'{FeatureName.FOLLOWING_POS.name}_{d[next_idx].pos_}'] += 1
                s_features[f'{FeatureName.FOLLOWING_POSTAG.name}_{d[next_idx].tag_}'] += 1
            # VERB_HEAD_DEP
            # VERB_HEAD_POS
            '''
            "Because the syntactic relations form a tree, every word has exactly one head.
            You can therefore iterate over the arcs in the tree by iterating over the words in the sentence."
            https://spacy.io/docs/usage/dependency-parse#navigating
            '''
            if (token.head is not token):
                s_features[f'{FeatureName.PARENT_DEP.name}_{token.head.dep_.upper()}'] += 1
                s_features[f'{FeatureName.PARENT_POS.name}_{token.head.pos_}'] += 1
                s_features[f'{FeatureName.PARENT_POSTAG.name}_{token.head.tag_}'] += 1
            # VERB_CHILD_DEP
            # VERB_CHILD_POS
            for child in token.children:
                s_features[f'{FeatureName.CHILD_DEP.name}_{child.dep_.upper()}'] += 1
                s_features[f'{FeatureName.CHILD_POS.name}_{child.pos_}'] += 1
                s_features[f'{FeatureName.CHILD_POSTAG.name}_{child.tag_}'] += 1
    return dict(s_features)
        #print(dict(s_features))
    #print()

#print(featuresets, len(featuresets))

In [15]:
featuresets = [(doc[0], (featurize(doc[0]), doc[1])) for doc in docs]

In [16]:
from statistics import mean, median, mode, stdev
f_lengths = [len(fs[1][0]) for fs in featuresets]

print('Stats on number of features per example:')
print(f'mean: {mean(f_lengths)}')
print(f'stdev: {stdev(f_lengths)}')
print(f'median: {median(f_lengths)}')
print(f'mode: {mode(f_lengths)}')
print(f'max: {max(f_lengths)}')
print(f'min: {min(f_lengths)}')

Stats on number of features per example:
mean: 23.019180470793373
stdev: 14.696807269292922
median: 23.0
mode: 0
max: 75
min: 0


In [17]:
featuresets[:2]

[(Divide mushrooms and chickpeas into separate piles and spread in a single layer.,
  ({'CHILD_DEP_NSUBJ': 1,
    'CHILD_DEP_PREP': 1,
    'CHILD_DEP_PUNCT': 1,
    'CHILD_POSTAG_.': 1,
    'CHILD_POSTAG_IN': 1,
    'CHILD_POSTAG_NNS': 1,
    'CHILD_POS_ADP': 1,
    'CHILD_POS_NOUN': 1,
    'CHILD_POS_PUNCT': 1,
    'FOLLOWING_POSTAG_IN': 1,
    'FOLLOWING_POS_ADP': 1,
    'VERB': 1},
   'pos')),
 (Add 1 Tbsp. oil, then sugar snap peas, and shake pan so sugar snaps spread out in a single layer.,
  ({'CHILD_DEP_ADVMOD': 1,
    'CHILD_DEP_APPOS': 1,
    'CHILD_DEP_CC': 1,
    'CHILD_DEP_CONJ': 2,
    'CHILD_DEP_DEP': 1,
    'CHILD_DEP_DOBJ': 2,
    'CHILD_DEP_NSUBJ': 1,
    'CHILD_DEP_NUMMOD': 1,
    'CHILD_DEP_PREP': 1,
    'CHILD_DEP_PRT': 1,
    'CHILD_DEP_PUNCT': 2,
    'CHILD_POSTAG_.': 2,
    'CHILD_POSTAG_CC': 1,
    'CHILD_POSTAG_CD': 1,
    'CHILD_POSTAG_IN': 1,
    'CHILD_POSTAG_NN': 2,
    'CHILD_POSTAG_NNP': 1,
    'CHILD_POSTAG_NNS': 2,
    'CHILD_POSTAG_RB': 1,
    'CHILD_P

On one run, the above line printed the following featureset:
`(Gather foil loosely on top and bake for 1 1/2 hours., ({}, 'pos'))`

This is because the Spacy.io POS tagger provided this:
   `Gather/NNP foil/NN loosely/RB on/IN top/NN and/CC bake/NN for/IN 1 1/2 hours./NNS`

With no VERBs tagged, which is incorrect.

---
Compare to [Stanford CoreNLP POS tagger](http://nlp.stanford.edu:8080/corenlp/process):
   `Gather/VB foil/NN loosely/RB on/IN top/JJ and/CC bake/VB for/IN 1 1/2/CD hours/NNS ./.`

And [Stanford Parser](http://nlp.stanford.edu:8080/parser/index.jsp):
   `Gather/NNP foil/VB loosely/RB on/IN top/NN and/CC bake/VB for/IN 1 1/2/CD hours/NNS ./.`

### Classification

In [18]:
random.shuffle(featuresets)

split_num = round(num / 5)

print(f'# training samples: {num-split_num}')
print(f'# test samples: {split_num}')

# train and test sets
testing_set = [fs[1] for i, fs in enumerate(featuresets[:split_num])]
training_set =  [fs[1] for i, fs in enumerate(featuresets[split_num:])]

# training samples: 1835
# test samples: 459


In [19]:
# decoupling the functionality of nltk.classify.accuracy
def predict(classifier, gold, prob=True):
    if (prob is True):
        predictions = classifier.prob_classify_many([fs for (fs, ll) in gold])
    else:
        predictions = classifier.classify_many([fs for (fs, ll) in gold])
    return list(zip(predictions, [ll for (fs, ll) in gold]))

def accuracy(predicts, prob=True):
    if (prob is True):
        correct = [label == prediction.max() for (prediction, label) in predicts]
    else:
        correct = [label == prediction for (prediction, label) in predicts]
        
    if correct:
        return sum(correct) / len(correct)
    else:
        return 0

Note below the use of `DummyClassifier` to provide a simple sanity check, a baseline of random predictions. `stratified` means it "generates random predictions by respecting the training set class distribution." (http://scikit-learn.org/stable/modules/model_evaluation.html#dummy-estimators)

> More generally, when the accuracy of a classifier is too close to random, it probably means that something went wrong: features are not helpful, a hyperparameter is not correctly tuned, the classifier is suffering from class imbalance, etc…

If a classifier can beat the `DummyClassifier`, it is at least learning something valuable! How valuable is another question...

In [20]:
from nltk import NaiveBayesClassifier
from nltk.classify.decisiontree import DecisionTreeClassifier
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

dummy = SklearnClassifier(DummyClassifier(strategy='stratified', random_state=0))
dummy.train(training_set)
dummy_predict = predict(dummy, testing_set)
dummy_accuracy = accuracy(dummy_predict)
print("Dummy classifier accuracy percent:", dummy_accuracy*100)

nb = NaiveBayesClassifier.train(training_set)
nb_predict = predict(nb, testing_set)
nb_accuracy = accuracy(nb_predict)
print("NaiveBayes classifier accuracy percent:", nb_accuracy*100)

multinomial_nb = SklearnClassifier(MultinomialNB())
multinomial_nb.train(training_set)
mnb_predict = predict(multinomial_nb, testing_set)
mnb_accuracy = accuracy(mnb_predict)
print("MultinomialNB classifier accuracy percent:", mnb_accuracy*100)

bernoulli_nb = SklearnClassifier(BernoulliNB())
bernoulli_nb.train(training_set)
bnb_predict = predict(bernoulli_nb, testing_set)
bnb_accuracy = accuracy(bnb_predict)
print("BernoulliNB classifier accuracy percent:", bnb_accuracy*100)

# ??logistic_regression._clf
#   sklearn.svm.LinearSVC : learns SVM models using the same algorithm.
logistic_regression = SklearnClassifier(LogisticRegression())
logistic_regression.train(training_set)
lr_predict = predict(logistic_regression, testing_set)
lr_accuracy = accuracy(lr_predict)
print("LogisticRegression classifier accuracy percent:", lr_accuracy*100)

# ??sgd._clf
#    The 'log' loss gives logistic regression, a probabilistic classifier.
# ??linear_svc._clf
#   can optimize the same cost function as LinearSVC
#   by adjusting the penalty and loss parameters. In addition it requires
#   less memory, allows incremental (online) learning, and implements
#   various loss functions and regularization regimes.
sgd = SklearnClassifier(SGDClassifier(loss='log'))
sgd.train(training_set)
sgd_predict = predict(sgd, testing_set)
sgd_accuracy = accuracy(sgd_predict)
print("SGD classifier accuracy percent:", sgd_accuracy*100)

# using libsvm with kernel 'rbf' (radial basis function)
svc = SklearnClassifier(SVC(probability=True))
svc.train(training_set)
svc_predict = predict(svc, testing_set)
svc_accuracy = accuracy(svc_predict)
print("SVC classifier accuracy percent:", svc_accuracy*100)

# ??linear_svc._clf
#    Similar to SVC with parameter kernel='linear', but implemented in terms of
#    liblinear rather than libsvm, so it has more flexibility in the choice of
#    penalties and loss functions and should scale better to large numbers of
#    samples.
#    Prefer dual=False when n_samples > n_features.
linear_svc = SklearnClassifier(LinearSVC(dual=False))
linear_svc.train(training_set)
linear_svc_predict = predict(linear_svc, testing_set, False)
linear_svc_accuracy = accuracy(linear_svc_predict, False)
print("LinearSVC classifier accuracy percent:", linear_svc_accuracy*100)

# slow
dt = DecisionTreeClassifier.train(training_set)
dt_predict = predict(dt, testing_set, False)
dt_accuracy = accuracy(dt_predict, False)
print("DecisionTree classifier accuracy percent:", dt_accuracy*100)

Dummy classifier accuracy percent: 52.28758169934641
NaiveBayes classifier accuracy percent: 63.61655773420479
MultinomialNB classifier accuracy percent: 80.3921568627451
BernoulliNB classifier accuracy percent: 76.25272331154684
LogisticRegression classifier accuracy percent: 85.18518518518519
SGD classifier accuracy percent: 74.7276688453159
SVC classifier accuracy percent: 84.31372549019608
LinearSVC classifier accuracy percent: 85.40305010893246
DecisionTree classifier accuracy percent: 79.08496732026144


### SGD: Multiple Epochs

`sgd` classifiers improves with epochs. `??sgd._clf` tells us that the default number of epochs `n_iter` is 5. So let's run more epochs. Also not that the training_set shuffle is `True` by default.

In [21]:
num_epochs = 1000
sgd = SklearnClassifier(SGDClassifier(loss='log', n_iter=num_epochs))
sgd.train(training_set)
sgd_predict = predict(sgd, testing_set)
sgd_accuracy = accuracy(sgd_predict)
print(f"SGDClassifier classifier accuracy percent (epochs: {num_epochs}):", sgd_accuracy*100)

SGDClassifier classifier accuracy percent (epochs: 1000): 84.9673202614379


Fortunately, 1000 epochs run very quickly! And `SGDClassifier` performance has improved with more iterations.

_Also note that we can set `warm_start` to `True` if we want to take advantage of online learning and reuse the solution of the previous call._

### Analysis

We're going to scope analysis down to our top-performing classifiers, which consistently perform with >80% accuracy: `LogisticRegression`, `SVC`, `LinearSVC`, and `SGD`.

We'll also include `Dummy` as a baseline.

**TODO**: Omit `LinearSVC`? Since we have `SVC` performing well, and `LinearSVC` does not provide probability estimates. (However, `LinearSVC` is meant to "scale better to large numbers of samples" - i.e., `LinearSVC` is faster, as I've witnessed)

#### Performance on sample tasks

In [22]:
sample_tasks = ["Mow lawn", "Mow the lawn", "Buy new shoes", "Feed the dog", "Send report to Kyle", "Send the report to Kyle", "Peel the potatoes"]
features = [featurize(nlp(task)) for task in sample_tasks]

tasks_dummy = [(l, p.prob('pos')*1.0) for l, p in zip(dummy.classify_many(features), dummy.prob_classify_many(features))]
tasks_logistic = [(l, p.prob('pos')) for l,p in zip(logistic_regression.classify_many(features), logistic_regression.prob_classify_many(features))]
tasks_svc = [(l, p.prob('pos')) for l,p in zip(svc.classify_many(features), svc.prob_classify_many(features))]
tasks_linear_svc = linear_svc.classify_many(features)
tasks_sgd = [(l, p.prob('pos')) for l,p in zip(sgd.classify_many(features), sgd.prob_classify_many(features))]

print(f'Dummy: {tasks_dummy}')
print(f'LogisticRegression: {tasks_logistic}')
print(f'SVC: {tasks_svc}')
print(f'LinearSVC: {tasks_linear_svc}')
print(f'SGD: {tasks_sgd}')

Dummy: [('pos', 1.0), ('pos', 1.0), ('pos', 1.0), ('pos', 1.0), ('neg', 0.0), ('pos', 1.0), ('neg', 0.0)]
LogisticRegression: [('pos', 0.57787315917563198), ('pos', 0.71976787805644704), ('pos', 0.90202359673281007), ('pos', 0.71976787805644704), ('pos', 0.87517196834667843), ('pos', 0.71976787805644704), ('pos', 0.74962733388797642)]
SVC: [('pos', 0.78337496136116336), ('pos', 0.7371123757061474), ('pos', 0.84387550274254164), ('pos', 0.7371123757061474), ('pos', 0.81730958890799943), ('pos', 0.7371123757061474), ('pos', 0.77492180934221044)]
LinearSVC: ['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
SGD: [('pos', 0.57554835689985495), ('pos', 0.73170163923160192), ('pos', 0.91605868461901896), ('pos', 0.73170163923160192), ('pos', 0.90172719947511659), ('pos', 0.73170163923160192), ('pos', 0.76426341375478135)]


_Note: `LinearSVC` is not implemented to provide probability estimates (https://github.com/scikit-learn/scikit-learn/issues/4820)_

**TODO** tabular format would be cool above

**TODO** Performance on prototype feedback

#### Most Informative Features

In [23]:
# https://stackoverflow.com/a/11140887
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:round(n/2)], coefs_with_fns[:-(round(n/2) + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [24]:
print('SGD')
show_most_informative_features(sgd._vectorizer, sgd._clf, 15)
print()
print('Logistic Regression')
show_most_informative_features(logistic_regression._vectorizer, logistic_regression._clf, 15)
print()
print('LinearSVC')
show_most_informative_features(linear_svc._vectorizer, linear_svc._clf, 15)

SGD
	-2.9453	CHILD_DEP_AGENT		2.5849	CHILD_POSTAG_-RRB-
	-2.5896	FOLLOWING_POSTAG_WRB		1.9425	PARENT_POS_PROPN
	-2.5486	PARENT_DEP_AMOD		1.9425	PARENT_POSTAG_NNP
	-2.4047	CHILD_DEP_INTJ 		1.7827	PARENT_DEP_ADVMOD||CONJ
	-2.3380	CHILD_POSTAG_``		1.7600	FOLLOWING_POSTAG_NFP
	-2.1528	FOLLOWING_POSTAG_-RRB-		1.6705	FOLLOWING_POSTAG_RBS
	-1.9480	CHILD_POSTAG_HYPH		1.6692	CHILD_DEP_ADVMOD||XCOMP
	-1.6555	CHILD_POSTAG_''		1.6173	CHILD_DEP_NPADVMOD

Logistic Regression
	-2.4040	CHILD_DEP_AGENT		2.0140	CHILD_POSTAG_-RRB-
	-1.7055	PARENT_DEP_AMOD		1.4831	CHILD_DEP_NPADVMOD
	-1.6710	CHILD_POSTAG_HYPH		1.4637	PARENT_POS_PROPN
	-1.6442	FOLLOWING_POSTAG_WRB		1.4637	PARENT_POSTAG_NNP
	-1.5683	CHILD_DEP_INTJ 		1.1580	VERB           
	-1.5185	CHILD_POSTAG_``		1.0738	CHILD_POS_PROPN
	-1.4930	CHILD_DEP_NSUBJ		1.0508	CHILD_POSTAG_NNP
	-1.4131	PARENT_POSTAG_VBZ		1.0454	CHILD_DEP_DOBJ||XCOMP

LinearSVC
	-1.4346	FOLLOWING_POSTAG_WRB		1.5444	PARENT_DEP_ADVMOD||CONJ
	-1.4069	CHILD_POSTAG_``		1.3265	FOLLOWING_P

*Note: Because `SVC` is using the nonlinear RBF kernel, we cannot show the most informative features (`coef_ is only available when using a linear kernel`).*

**TODO** Provide meaning behind these features

#### Scikit Learn metrics: Confusion matrix, Classification report, F1 score, Log loss

http://scikit-learn.org/stable/modules/model_evaluation.html

In [25]:
from sklearn import metrics

def classification_report(predict, prob=True):
    predictions, labels = zip(*predict)
    if prob is True:
        return metrics.classification_report(labels, [p.max() for p in predictions])
    else:
        return metrics.classification_report(labels, predictions)

def confusion_matrix(predict, prob=True, print_layout=False):
    predictions, labels = zip(*predict)
    if print_layout is True:
        print('Layout\n[[tn   fp]\n [fn   tp]]\n')
    if prob is True:
        return metrics.confusion_matrix(labels, [p.max() for p in predictions])
    else:
        return metrics.confusion_matrix(labels, predictions)

def log_loss(predict):
    predictions, labels = zip(*predict)
    return metrics.log_loss(labels, [p.prob('pos') for p in predictions])

def roc_auc_score(predict):
    predictions, labels = zip(*predict)
    # need to convert labels to binary classification of 0 or 1
    return metrics.roc_auc_score([1 if l == 'pos' else 0 for l in labels], [p.prob('pos') for p in predictions], average='weighted')

In [26]:
print('SGD')
print(classification_report(sgd_predict))
print()
print('Logistic Regression')
print(classification_report(lr_predict))
print()
print('SVC')
print(classification_report(svc_predict))
print()
print('LinearSVC')
print(classification_report(linear_svc_predict, False))

SGD
             precision    recall  f1-score   support

        neg       0.89      0.79      0.84       226
        pos       0.82      0.91      0.86       233

avg / total       0.85      0.85      0.85       459


Logistic Regression
             precision    recall  f1-score   support

        neg       0.90      0.78      0.84       226
        pos       0.81      0.92      0.86       233

avg / total       0.86      0.85      0.85       459


SVC
             precision    recall  f1-score   support

        neg       0.87      0.80      0.83       226
        pos       0.82      0.89      0.85       233

avg / total       0.85      0.84      0.84       459


LinearSVC
             precision    recall  f1-score   support

        neg       0.90      0.79      0.84       226
        pos       0.82      0.92      0.86       233

avg / total       0.86      0.85      0.85       459



In [27]:
print('Layout\n[[tn   fp]\n [fn   tp]]\n')

print('SGD')
print(confusion_matrix(sgd_predict))
print()
print('Logistic Regression')
print(confusion_matrix(lr_predict))
print()
print('SVC')
print(confusion_matrix(svc_predict))
print()
print('LinearSVC')
print(confusion_matrix(linear_svc_predict, False))

Layout
[[tn   fp]
 [fn   tp]]

SGD
[[178  48]
 [ 21 212]]

Logistic Regression
[[177  49]
 [ 19 214]]

SVC
[[180  46]
 [ 26 207]]

LinearSVC
[[178  48]
 [ 19 214]]


The lower the better for `log_loss`...

In [28]:
print(f'SGD: {log_loss(sgd_predict)}')
print(f'Logistic Regression: {log_loss(lr_predict)}')
print(f'SVC: {log_loss(svc_predict)}')

SGD: 0.35720755386910946
Logistic Regression: 0.35259513950041504
SVC: 0.3785530776608165


The higher the better for `roc_auc_score`...

In [29]:
print(f'SGD: {roc_auc_score(sgd_predict)}')
print(f'Logistic Regression: {roc_auc_score(lr_predict)}')
print(f'SVC: {roc_auc_score(svc_predict)}')

SGD: 0.9199361920316002
Logistic Regression: 0.9223479813133808
SVC: 0.9070986364844847


*Note: We cannot compute `log_loss` or `roc_auc_score` for `LinearSVC` because it does not provide probability estimates.*

**Next up**: digging into the results (confusion matrix, most informative features), comparing results to LUIS model, cross validation/grid search

## Next Steps and Improvements

1. Training set may be too specific/not relevant enough (recipe instructions for positive dataset, recipe descriptions+short movie reviews for negative dataset)
2. Throwing features into a blender - need to understand value of each
3. Need to review different classifiers, strengths/weaknesses
4. Phrase vectorizations of all 0s
5. Varying feature vector lengths
6. Voting
7. Reducing dimensionality using most informative feature information
8. Combining verb phrases

# Things abandoned

## NLTK

I needed a library that supports dependency parsing, which NLTK does not... so I thought I'd add the [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/) toolkit and [its associated software](https://nlp.stanford.edu/software/) to NLTK. However, there are many conflicting instructions for installing the Java-based project, depending on NLTK version used. By the time I figured this out, the installation had become a time sink. So I abandoned this effort in favor of Spacy.io.

I might return this way if I want to improve results/implement a voter system between the various linguistic and classification methods later.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

### Tokenization

In [ ]:
sentences = [s for l in lines for s in sent_tokenize(l)] # punkt
sentences

In [ ]:
tagged_sentences = []
for s in sentences:
    words = word_tokenize(s)
    tagged = nltk.pos_tag(words) # averaged_perceptron_tagger
    tagged_sentences.append(tagged)
print(tagged_sentences)

#### Note: POS accuracy

`Run down to the shop, will you, Peter` is parsed unexpectedly by `nltk.pos_tag`:
> `[('Run', 'NNP'), ('down', 'RB'), ('to', 'TO'), ('the', 'DT'), ('shop', 'NN'), (',', ','), ('will', 'MD'), ('you', 'PRP'), (',', ','), ('Peter', 'NNP')]`

`Run` is tagged as a `NNP (proper noun, singular)`

I expected an output more like what the [Stanford Parser](http://nlp.stanford.edu:8080/parser/) provides:
> `Run/VBG down/RP to/TO the/DT shop/NN ,/, will/MD you/PRP ,/, Peter/NNP`

`Run` is tagged as a `VGB (verb, gerund/present participle)` - still not quite the `VB` I want, but at least it's a `V*`

_MEANWHILE..._

`nltk.pos_tag` did better with:
> `[('Do', 'VB'), ('not', 'RB'), ('clean', 'VB'), ('soot', 'NN'), ('off', 'IN'), ('the', 'DT'), ('window', 'NN')]`

Compared to [Stanford CoreNLP](http://nlp.stanford.edu:8080/corenlp/process) (note that this is different than what [Stanford Parser](http://nlp.stanford.edu:8080/parser/) outputs):
> `(ROOT (S (VP (VB Do) (NP (RB not) (JJ clean) (NN soot)) (PP (IN off) (NP (DT the) (NN window))))))`

Concern: _clean_ as `VB (verb, base form)` vs `JJ (adjective)` 

**IMPROVE** POS taggers should vote: nltk.pos_tag (averaged_perceptron_tagger), Stanford Parser, CoreNLP, etc.

Note what Spacy POS tagger did with `Run down to the shop, will you Peter`:

`Run/VB down/RP to/IN the shop/NN ,/, will/MD you/PRP ,/, Peter/NNP`

    where `Run` is the `VB` I expected from POS tagging (compared to `nltk.pos_tag` result of `NNP`). Also note that Spacy collapses `the shop` into a single unit, which should be helpful during featurization.

### Featurization

In [ ]:
import re
from collections import defaultdict

featuresets = []
for ts in tagged_sentences:
    s_features = defaultdict(int)
    for idx, tup in enumerate(ts):
        #print(tup)
        pos = tup[1]
        # FeatureName.VERB
        is_verb = re.match(r'VB.?', pos) is not None
        print(tup, is_verb)
        if is_verb:
            s_features[FeatureName.VERB] += 1
            # FOLLOWING_POS
            next_idx = idx + 1;
            if next_idx < len(ts):
                s_features[f'{FeatureName.FOLLOWING}_{ts[next_idx][1]}'] += 1
            # VERB_MODIFIER
            # VERB_MODIFYING
        else:
            s_features[FeatureName.VERB] = 0
    featuresets.append(dict(s_features))

print()
print(featuresets)

### [Stanford NLP](https://nlp.stanford.edu/software/)
Setup guide used: https://stackoverflow.com/a/34112695

In [ ]:
# Get dependency parser, NER, POS tagger
!wget https://nlp.stanford.edu/software/stanford-parser-full-2017-06-09.zip
!wget https://nlp.stanford.edu/software/stanford-ner-2017-06-09.zip
!wget https://nlp.stanford.edu/software/stanford-postagger-full-2017-06-09.zip
!unzip stanford-parser-full-2017-06-09.zip
!unzip stanford-ner-2017-06-09.zip
!unzip stanford-postagger-full-2017-06-09.zip

In [ ]:
from nltk.parse.stanford import StanfordParser
from nltk.parse.stanford import StanfordDependencyParser
from nltk.parse.stanford import StanfordNeuralDependencyParser
from nltk.tag.stanford import StanfordPOSTagger, StanfordNERTagger
from nltk.tokenize.stanford import StanfordTokenizer